# Pipeline for High-z Radio Galaxies 06: Application of full pipeline for prediction with optimised thresholds

## Introduction

In this file, three models will be applied consecutively in order to predict  
the detection of Radio Galaxies (radio AGN) and their redshift.  

In principle, this pipeline should be applied to data in Stripe 82. But  
it can be used with any other suitable dataset.

In [1]:
%matplotlib inline
# Static plots
#%matplotlib ipympl
# Interactive plots
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib import ticker
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patheffects as mpe
import matplotlib.patches as mpatches
from matplotlib.ticker import ScalarFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.ndimage import gaussian_filter
from astropy import units as u
from astropy.cosmology import Planck18 as cosmo
from astropy.visualization import LogStretch, PowerStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
import sklearn.pipeline
import colorcet as cc
import cmasher as cmr
from pycaret import classification as pyc
from pycaret import regression as pyr
import pandas as pd
import mpl_scatter_density
from joblib import dump, load
import global_variables as gv
import global_functions as gf

In [2]:
mpl.rcdefaults()

In [3]:
plt.rcParams['text.usetex'] = True

Functions to predict values

In [4]:
def predict_star(catalog_df, star_model, cal_str_model, threshold, cal_threshold, raw_score=True):
    catalog_df = pyc.predict_model(star_model, data=catalog_df, probability_threshold=threshold, raw_score=raw_score, round=10)
    catalog_df = catalog_df.drop(columns=['Score_1'])
    catalog_df = catalog_df.rename(columns={'Label': 'pred_star', 'Score_0': 'Score_no_star'})
    catalog_df.loc[:, 'Score_no_star'] = np.around(catalog_df.loc[:, 'Score_no_star'], decimals=7)
    pred_probs = cal_str_model.predict(catalog_df.loc[:, 'Score_0'])
    cal_class  = np.array(pred_probs < (1 - cal_threshold)).astype(int)
    catalog_df['Prob_no_star']  = pred_probs
    catalog_df['pred_star_cal'] = cal_class
    return catalog_df

In [5]:
def predict_AGN_gal(catalog_df, AGN_gal_model, cal_AGN_gal_model, threshold, cal_threshold, raw_score=True):
    catalog_df = pyc.predict_model(AGN_gal_model, data=catalog_df, probability_threshold=threshold, raw_score=raw_score, round=10)
    catalog_df = catalog_df.drop(columns=['Score_0'])
    catalog_df = catalog_df.rename(columns={'Label': 'pred_class', 'Score_1': 'Score_AGN'})
    catalog_df.loc[:, 'Score_AGN'] = np.around(catalog_df.loc[:, 'Score_AGN'], decimals=7)
    pred_probs = cal_AGN_gal_model.predict(catalog_df.loc[:, 'Score_AGN'])
    cal_class  = np.array(pred_probs >= cal_threshold).astype(int)
    catalog_df['Prob_AGN']       = pred_probs
    catalog_df['pred_class_cal'] = cal_class
    return catalog_df

In [6]:
def predict_radio_det(catalog_df, radio_model, cal_radio_model, threshold, cal_threshold, raw_score=True):
    catalog_df = pyc.predict_model(radio_model, data=catalog_df, probability_threshold=threshold, raw_score=raw_score, round=10)
    catalog_df = catalog_df.drop(columns=['Score_0'])
    catalog_df = catalog_df.rename(columns={'Label': 'pred_radio', 'Score_1': 'Score_radio'})
    catalog_df.loc[:, 'Score_radio'] = np.around(catalog_df.loc[:, 'Score_radio'], decimals=7)
    pred_probs = cal_radio_model.predict(catalog_df.loc[:, 'Score_radio'])
    cal_class  = np.array(pred_probs >= cal_threshold).astype(int)
    catalog_df['Prob_radio']     = pred_probs
    catalog_df['pred_radio_cal'] = cal_class
    return catalog_df

In [7]:
def predict_z_full(catalog_df, redshift_model):
    catalog_df = pyr.predict_model(redshift_model, data=catalog_df, round=10)
    catalog_df = catalog_df.rename(columns={'Label': 'pred_Z'})
    catalog_df.loc[:, 'pred_Z'] = np.around(catalog_df.loc[:, 'pred_Z'], decimals=4)
    return catalog_df

In [8]:
def predict_z_high(catalog_df, redshift_model, z_lim, z_tol):
    catalog_df    = pyr.predict_model(redshift_model, data=catalog_df, round=10)
    filter_pred_z = catalog_df.loc[:, 'pred_Z'] >= (z_lim + z_tol)
    catalog_df.loc[:, 'pred_Z'] = catalog_df.loc[:, 'pred_Z'].mask(filter_pred_z, catalog_df.loc[filter_pred_z, 'Label'])
    catalog_df    = catalog_df.drop(columns=['Label'])
    catalog_df.loc[:, 'pred_Z'] = np.around(catalog_df.loc[:, 'pred_Z'], decimals=4)
    return catalog_df

In [9]:
def M12_AGN_criterion(catalog_df):
    M12_column = (np.array(catalog_df.loc[:, 'W1mproPM'] - catalog_df.loc[:, 'W2mproPM'] - 2.699 + 3.339 <
                           0.315 * (catalog_df.loc[:, 'W2mproPM'] - catalog_df.loc[:, 'W3mag'] - 3.339 + 5.174) + 0.791) &
                  np.array(catalog_df.loc[:, 'W1mproPM'] - catalog_df.loc[:, 'W2mproPM'] - 2.699 + 3.339 >
                           0.315 * (catalog_df.loc[:, 'W2mproPM'] - catalog_df.loc[:, 'W3mag'] - 3.339 + 5.174) - 0.222) &
                  np.array(catalog_df.loc[:, 'W1mproPM'] - catalog_df.loc[:, 'W2mproPM'] - 2.699 + 3.339 >
                           -3.172 * (catalog_df.loc[:, 'W2mproPM'] - catalog_df.loc[:, 'W3mag'] - 3.339 + 5.174) + 7.624)).astype(int)
    return M12_column

In [10]:
def S12_AGN_criterion(catalog_df):
    S12_column = np.array(catalog_df.loc[:, 'W1mproPM'] - catalog_df.loc[:, 'W2mproPM'] - 2.699 + 3.339 >= 0.8).astype(int)
    return S12_column

In [11]:
def M16_AGN_criterion(catalog_df):
    M16_column = (np.array(catalog_df.loc[:, 'W1mproPM'] - catalog_df.loc[:, 'W2mproPM'] - 2.699 + 3.339 > 0.5) &
                  np.array(catalog_df.loc[:, 'W2mproPM'] - catalog_df.loc[:, 'W3mag'] - 3.339 + 5.174 < 4.4)).astype(int)
    return M16_column

In [12]:
def B18_AGN_criterion(catalog_df):
    B18_column = (np.array(catalog_df.loc[:, 'W1mproPM'] - catalog_df.loc[:, 'W2mproPM'] - 2.699 + 3.339 > 0.5) &
                  np.array(catalog_df.loc[:, 'W2mproPM'] - catalog_df.loc[:, 'W3mag'] - 3.339 + 5.174 > 2.2) &
                  np.array(catalog_df.loc[:, 'W1mproPM'] - catalog_df.loc[:, 'W2mproPM'] - 2.699 + 3.339 >
                           2 * (catalog_df.loc[:, 'W2mproPM'] - catalog_df.loc[:, 'W3mag'] - 3.339 + 5.174) - 8.9)).astype(int)
    return B18_column

In [13]:
def C22_AGN_criterion(catalog_df):  # This work
    # C22_column = (np.array(catalog_df.loc[:, 'rmag'] - catalog_df.loc[:, 'zmag'] - 0.617 + 0.866 > -0.45) &
    #               np.array(catalog_df.loc[:, 'rmag'] - catalog_df.loc[:, 'zmag'] - 0.617 + 0.866 < 1.8) &
    #               np.array(catalog_df.loc[:, 'W1mproPM'] - catalog_df.loc[:, 'W2mproPM'] - 2.699 + 3.339 >
    #                        0.355 * (catalog_df.loc[:, 'rmag'] - catalog_df.loc[:, 'zmag'] - 0.617 + 0.866) + 0.26)).astype(int)  # W1-W2, r-z
    C22_column = (np.array(catalog_df.loc[:, 'gmag'] - catalog_df.loc[:, 'rmag'] - 0.481 + 0.617 > -0.76) &
                  np.array(catalog_df.loc[:, 'gmag'] - catalog_df.loc[:, 'rmag'] - 0.481 + 0.617 < 1.8) &
                  np.array(catalog_df.loc[:, 'W1mproPM'] - catalog_df.loc[:, 'W2mproPM'] - 2.699 + 3.339 >
                           0.227 * (catalog_df.loc[:, 'rmag'] - catalog_df.loc[:, 'zmag'] - 0.481 + 0.617) + 0.43)).astype(int)  # W1-W2, g-r
    return C22_column

In [14]:
def add_AGN_criteria(catalog_df):
    catalog_df['M12_AGN'] = M12_AGN_criterion(catalog_df)
    catalog_df['S12_AGN'] = S12_AGN_criterion(catalog_df)
    catalog_df['M16_AGN'] = M16_AGN_criterion(catalog_df)
    catalog_df['B18_AGN'] = B18_AGN_criterion(catalog_df)
    catalog_df['C22_AGN'] = C22_AGN_criterion(catalog_df)
    return catalog_df

In [15]:
def radio_lum(flux, z, alpha):  # Output in W Hz-1
    lum_dist = cosmo.luminosity_distance(z).to(u.m)
    num      = 4 * np.pi * flux.to(u.Jy) * lum_dist**2
    denom    = (1 + z)**(alpha +1)
    return (num/denom).to(u.W / u.Hz)

In [16]:
def cut_rgb_val(val):
    if val < 0.0:
        return 0.0
    if val > 1.0:
        return 1.0
    else:
        return val

In [17]:
def fmt(x):
    x = x * 100.
    x = 100. - x
    s = f'{x:.2f}'
    if s.endswith('0'):
        s = f'{x:.0f}'
    return rf'{s} \%' if plt.rcParams['text.usetex'] else f'{s} %'

In [18]:
def make_borders_zero(matrix):
    matrix[:, 0]  = 0.0
    matrix[:, -1] = 0.0
    matrix[0, :]  = 0.0
    matrix[-1, :] = 0.0
    return matrix

In [19]:
def pad_matrix_zeros(matrix, xedges, yedges):  # Pads matrices and creates centred edges
    x_centres = 0.5 * (xedges[:-1] + xedges[1:])
    y_centres = 0.5 * (yedges[:-1] + yedges[1:])
    matrix    = np.pad(matrix, ((1, 1), (1, 1)), mode='constant', constant_values=(0,))
    x_centres = np.pad(x_centres, (1, 1), mode='constant', constant_values=(xedges[0], xedges[-1]))
    y_centres = np.pad(y_centres, (1, 1), mode='constant', constant_values=(yedges[0], yedges[-1]))
    return matrix, x_centres, y_centres

In [20]:
def clean_and_smooth_matrix(matrix, sigma=0.9):
    matrix[~np.isfinite(matrix)] = 0
    matrix_smooth = gaussian_filter(matrix, sigma=0.9)
    matrix_smooth[~np.isfinite(matrix_smooth)] = 0
    return matrix_smooth

In [21]:
colour_hex_rAGN        = '#D32F2F'
colour_cet_rAGN        = plt.get_cmap('cet_rainbow')(1.0)
colour_rAGN            = mcolors.to_rgba(colour_cet_rAGN)
colour_rAGN_rgb        = mcolors.to_rgb(colour_cet_rAGN)
colour_rAGN_shade      = list(colour_rAGN)
colour_rAGN_shade[3]   = 0.75
colour_rAGN_shade      = tuple(colour_rAGN_shade)
colour_rAGN_rgb_darker = list(colour_rAGN_rgb)
colour_rAGN_rgb_darker = list([value * 0.7 for value in colour_rAGN_rgb_darker])
colour_rAGN_rgb_bright = list([cut_rgb_val(value * 1.5) for value in list(colour_rAGN_rgb)])
colour_rAGN_rgb_darker = tuple(colour_rAGN_rgb_darker)
colour_rAGN_rgb_bright = tuple(colour_rAGN_rgb_bright)
colors_rAGN            = [colour_rAGN_rgb_darker, colour_rAGN_rgb_bright] # first color is darker
cm_gradient_rAGN       = mcolors.LinearSegmentedColormap.from_list('gradient_rAGN', colors_rAGN, N=50)

---

## Reading data

Flags.

In [22]:
save_plot_flag      = False
load_models_flag    = True
use_calibration     = True
compare_A17_flag    = True  # Compare with the results from Ananna et al., 2017
save_indices_flag   = False

In [23]:
used_area           = 'HETDEX'  # can be 'S82', 'HETDEX', 'COSMOS'
HETDEX_subset       = 'Validation'  # Validation, Training, Test, Test+Train, 'Calibration'
if used_area != 'HETDEX':
    HETDEX_subset   = ''

In [24]:
if used_area != 'S82':
    compare_A17_flag = False
if used_area != 'HETDEX':
    metrics_hiz_AGN = False

In [25]:
radio_alpha         = -0.7

In [26]:
if used_area == 'HETDEX':
    int_flux_col        = 'Sint_LOFAR'
    limit_flux_survey   = 71e-6 * 1e-3  # mJy
    limit_flux_survey_b = 71e-6  # 25 uJy
    radio_freq          = 150 * u.MHz
if used_area == 'S82':
    int_flux_col        = 'Fint_VLAS82'
    limit_flux_survey   = 52e-6 * 1e-3  # mJy
    limit_flux_survey_b = 52e-6  # 25 uJy
    radio_freq          = 1.4 * u.GHz

In [27]:
file_name_dict      = {'S82': gv.file_S82, 'HETDEX': gv.file_HETDEX, 'COSMOS': gv.file_COSMOS}
file_name           = file_name_dict[used_area]

In [28]:
feats_2_disc_S82    = ['RA_MILLI', 'DEC_MILLI', 'W1mag', 'W2mag', 'num_imputed', 'radio_detect']
feats_2_disc_HETDEX = ['RA_MILLI', 'DEC_MILLI', 'W1mag', 'W2mag', 'num_imputed', 'radio_detect']
feats_2_disc_COSMOS = ['RA_MILLI', 'DEC_MILLI', 'W1mag', 'W2mag', 'num_imputed', 'radio_detect', ]

feats_2_disc        = {'S82': feats_2_disc_S82, 'HETDEX': feats_2_disc_HETDEX, 'COSMOS': feats_2_disc_COSMOS}
features_2_discard  = feats_2_disc[used_area]

In [29]:
full_catalog_df     = pd.read_hdf(gv.cat_path + file_name, key='df').drop(columns=features_2_discard)

In [30]:
if used_area == 'S82':
    full_catalog_df.loc[:, 'LOFAR_detect'] = full_catalog_df.loc[:, 'VLAS82_detect'].copy()
    full_catalog_df = full_catalog_df.drop(columns=['VLAS82_detect'])
if used_area == 'COSMOS':
    full_catalog_df.loc[:, 'LOFAR_detect'] = full_catalog_df.loc[:, 'COSMOSVLA3_detect'].copy()
    full_catalog_df = full_catalog_df.drop(columns=['COSMOSVLA3_detect'])

Create features with class and combined redshift.

In [31]:
full_catalog_df['class']            = full_catalog_df.loc[:, 'is_AGN'].copy()
filter_non_confirmed                = np.array(full_catalog_df.loc[:, 'is_AGN'] == 1) |\
                                      np.array(full_catalog_df.loc[:, 'is_gal'] == 1)
full_catalog_df.loc[~filter_non_confirmed, 'class'] = 0.5
idx_non_Z                           = full_catalog_df.loc[:, 'Z'].where(full_catalog_df.loc[:, 'Z'] > 0).isna()
full_catalog_df.loc[idx_non_Z, 'Z'] = full_catalog_df.loc[:, 'Z'].mask(idx_non_Z, full_catalog_df.loc[idx_non_Z, 'zsp'])

Create column for detection as Radio AGN

In [32]:
full_catalog_df['radio_AGN']        = np.array(full_catalog_df.loc[:, 'is_AGN'] == 1) & np.array(full_catalog_df.loc[:, 'LOFAR_detect'] == 1)
full_catalog_df['radio_gal']        = np.array(full_catalog_df.loc[:, 'is_gal'] == 1) & np.array(full_catalog_df.loc[:, 'LOFAR_detect'] == 1)

Show early statistics of confirmed sources

In [33]:
print(f'In the {used_area} field, there are {np.sum(full_catalog_df.loc[:, "class"] != 0.5):,} confirmed sources.')
print(f'{np.sum(full_catalog_df.loc[:, "class"] == 0):,} of them are galaxies.')
print(f'{np.sum(full_catalog_df.loc[:, "class"] == 1):,} of them are AGN.')
print(f'{np.sum(full_catalog_df.loc[:, "class"] == 2):,} of them are stars.')

In the HETDEX field, there are 118,734 confirmed sources.
68,196 of them are galaxies.
50,538 of them are AGN.
0 of them are stars.


---

Split dataset if from HETDEX

In [34]:
if used_area == 'HETDEX':
    filter_known_spec = (full_catalog_df.loc[:, 'is_AGN'] == 1) | (full_catalog_df.loc[:, 'is_gal'] == 1)
    unknown_cat_df    = full_catalog_df.loc[~filter_known_spec]
    full_catalog_df   = full_catalog_df.loc[filter_known_spec]
    train_test_df, train_df, test_df, calibration_df, validation_df = gf.split_set(full_catalog_df, [0.2, 0.2, 0.5],\
                                                                               'is_AGN', use_calibration=use_calibration)
    # Save indices of sources per sub-set
    if save_indices_flag:
        idx_train_test  = train_test_df.index.to_numpy()
        idx_train       = train_df.index.to_numpy()
        idx_test        = test_df.index.to_numpy()
        idx_calibration = calibration_df.index.to_numpy()
        idx_validation  = validation_df.index.to_numpy()
        
        np.savetxt(gv.indices_path + 'indices_train_test.txt',  idx_train_test,  fmt='%d')
        np.savetxt(gv.indices_path + 'indices_train.txt',       idx_train,       fmt='%d')
        np.savetxt(gv.indices_path + 'indices_test.txt',        idx_test,        fmt='%d')
        np.savetxt(gv.indices_path + 'indices_calibration.txt', idx_calibration, fmt='%d')
        np.savetxt(gv.indices_path + 'indices_validation.txt',  idx_validation,  fmt='%d')
    
    print('Shape of used data in HETDEX')
    print('-' * 65)
    print(f'Full confirmed dataset size:                      {full_catalog_df.shape}')
    print(f'Data for Modeling (Train, Test, and Calibration): {train_test_df.shape}')
    print(f'Training data:                                    {train_df.shape}')
    print(f'Testing data:                                     {test_df.shape}')
    if use_calibration:
        print(f'Calibration data:                                 {calibration_df.shape}')
    print(f'Validation data:                                  {validation_df.shape}')
    print('-' * 65)
    print()
    print(f'Using {HETDEX_subset} data from HETDEX')
    selected_dataset = {'Training': train_df, 'Test': test_df, 'Test+Train': train_test_df,\
                        'Validation': validation_df, 'Calibration': calibration_df}
    known_catalog_df = selected_dataset[HETDEX_subset]

Shape of used data in HETDEX
-----------------------------------------------------------------
Full confirmed dataset size:                      (118734, 103)
Data for Modeling (Train, Test, and Calibration): (94987, 103)
Training data:                                    (75989, 103)
Testing data:                                     (9499, 103)
Calibration data:                                 (9499, 103)
Validation data:                                  (23747, 103)
-----------------------------------------------------------------

Using Validation data from HETDEX


Split data if not from HETDEX

In [35]:
if used_area != 'HETDEX':
    # filter_confirmed = (known_catalog_df.loc[:, 'class'] == 0) |\
    #                    (known_catalog_df.loc[:, 'class'] == 1) |\
    #                    (known_catalog_df.loc[:, 'class'] == 2)  # Galaxy, AGN, star
    filter_confirmed = (known_catalog_df.loc[:, 'class'] == 0) |\
                       (known_catalog_df.loc[:, 'class'] == 1) # Galaxy, AGN
    unknown_cat_df   = known_catalog_df.loc[~filter_confirmed]
    full_catalog_df  = known_catalog_df.loc[filter_confirmed]

In [36]:
all_subsets_dfs      = [known_catalog_df, unknown_cat_df]

In [37]:
print(f'Full used dataset size:              {known_catalog_df.shape}')
print('-' * 50)
print(f'Thus, it has {known_catalog_df.shape[0]:,} sources and {known_catalog_df.shape[1]:,} features.')

Full used dataset size:              (23747, 103)
--------------------------------------------------
Thus, it has 23,747 sources and 103 features.


Discard minor features.

In [38]:
full_catalog_df                     = known_catalog_df.drop(columns=['is_AGN', 'is_SDSS_QSO', 'is_SDSS_gal', 'is_gal', 'zsp', 'spCl'])

---

### Load models

In [39]:
if load_models_flag:
    AGN_gal_clf           = pyc.load_model(gv.models_path + gv.AGN_gal_model)  #  AGN/galaxy
    cal_AGN_gal_clf       = load(gv.models_path + gv.cal_AGN_gal_model)  # AGN/galaxy calibrated model
    radio_det_AGN_clf     = pyc.load_model(gv.models_path + gv.radio_model)  # Radio detection for AGN
    cal_radio_det_AGN_clf = load(gv.models_path + gv.cal_radio_model)  # calibrated model radio detection for AGN
    
    radio_det_gal_clf     = pyc.load_model(gv.models_path + gv.radio_galaxies_model)  # Radio detection for galaxies
    cal_radio_det_gal_clf = load(gv.models_path + gv.cal_radio_gals_model)  # calibrated model radio detection for galaxie
    
    redshift_reg_rAGN     = pyr.load_model(gv.models_path + gv.full_z_model)  # Redshift prediction for radio-AGN
    redshift_reg_rGal     = pyr.load_model(gv.models_path + gv.z_radio_galaxies_model)  # Redshift prediction for radio-galaxies

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


Predictions with optimised thresholds.

In [40]:
threshold_AGN_logit      = np.log(gv.cal_AGN_thresh   / (1 - gv.cal_AGN_thresh))
threshold_radio_logit    = np.log(gv.cal_radio_thresh / (1 - gv.cal_radio_thresh))

In [41]:
if load_models_flag:
    print('Predicting AGN/SFG in known dataset')
    known_catalog_df = gf.predict_AGN_gal(known_catalog_df,
                                            AGN_gal_clf,
                                            cal_AGN_gal_clf,
                                            gv.AGN_thresh,
                                            gv.cal_AGN_thresh)
    print('Predicting AGN/SFG in unknown dataset')
    # unknown_cat_df = gf.predict_AGN_gal(unknown_cat_df,
    #                                         AGN_gal_clf,
    #                                         cal_AGN_gal_clf,
    #                                         gv.AGN_thresh,
    #                                         gv.cal_AGN_thresh)

Predicting AGN/SFG in known dataset
Predicting AGN/SFG in unknown dataset


In [42]:
if load_models_flag:
    print('Predicting rAGN in known dataset')
    known_catalog_df = gf.predict_radio_det(known_catalog_df,
                                              radio_det_AGN_clf,
                                              cal_radio_det_AGN_clf,
                                              gv.radio_thresh,
                                              gv.cal_radio_thresh)
    known_catalog_df = known_catalog_df.rename(columns={'Score_radio': 'Score_radio_AGN',
                                                        'pred_radio': 'pred_radio_AGN',
                                                        'Prob_radio': 'Prob_radio_AGN',
                                                        'pred_radio_cal': 'pred_radio_cal_AGN'})
    print('Predicting rAGN in unknown dataset')
    # unknown_cat_df = gf.predict_radio_det(unknown_cat_df,
    #                                           radio_det_AGN_clf,
    #                                           cal_radio_det_AGN_clf,
    #                                           gv.radio_thresh,
    #                                           gv.cal_radio_thresh)
    # unknown_cat_df = unknown_cat_df.rename(columns={'Score_radio': 'Score_radio_AGN',
    #                                                     'pred_radio': 'pred_radio_AGN',
    #                                                     'Prob_radio': 'Prob_radio_AGN',
    #                                                     'pred_radio_cal': 'pred_radio_cal_AGN'})

Predicting rAGN in known dataset
Predicting rAGN in unknown dataset


In [43]:
if load_models_flag:
    print('Predicting rSFG in known dataset')
    known_catalog_df = gf.predict_radio_det(known_catalog_df,
                                              radio_det_gal_clf,
                                              cal_radio_det_gal_clf,
                                              gv.radio_gals_thresh,
                                              gv.cal_radio_gals_thresh)
    known_catalog_df = known_catalog_df.rename(columns={'Score_radio': 'Score_radio_gal',
                                                        'pred_radio': 'pred_radio_gal',
                                                        'Prob_radio': 'Prob_radio_gal',
                                                        'pred_radio_cal': 'pred_radio_cal_gal'})
    print('Predicting rSFG in unknown dataset')
    # unknown_cat_df = gf.predict_radio_det(unknown_cat_df,
    #                                           radio_det_gal_clf,
    #                                           cal_radio_det_gal_clf,
    #                                           gv.radio_gals_thresh,
    #                                           gv.cal_radio_gals_thresh)
    # unknown_cat_df = unknown_cat_df.rename(columns={'Score_radio': 'Score_radio_gal',
    #                                                     'pred_radio': 'pred_radio_gal',
    #                                                     'Prob_radio': 'Prob_radio_gal',
    #                                                     'pred_radio_cal': 'pred_radio_cal_gal'})

Predicting rSFG in known dataset
Predicting rSFG in unknown dataset


In [44]:
known_catalog_df.loc[:, 'pred_rAGN_cal'] = (np.array(known_catalog_df.loc[:, 'pred_class_cal'] == 1) & np.array(known_catalog_df.loc[:, 'pred_radio_cal_AGN'] == 1)).astype(int)
known_catalog_df.loc[:, 'pred_rGal_cal'] = (np.array(known_catalog_df.loc[:, 'pred_class_cal'] == 0) & np.array(known_catalog_df.loc[:, 'pred_radio_cal_gal'] == 1)).astype(int)

In [45]:
known_catalog_df.loc[:, 'class_gal']          = 1 - known_catalog_df.loc[:, 'class']
known_catalog_df.loc[:, 'pred_class_gal_cal'] = 1 - known_catalog_df.loc[:, 'pred_class_cal']

In [46]:
if load_models_flag:
    print('Predicting redshift for rAGN in known dataset')
    known_catalog_df = gf.predict_z(known_catalog_df, redshift_reg_rAGN)
    known_catalog_df = known_catalog_df.rename(columns={'pred_Z': 'pred_Z_rAGN'})
    print('Predicting redshift for rAGN in unknown dataset')
    # unknown_cat_df = gf.predict_z(unknown_cat_df, redshift_reg_rAGN)
    # unknown_cat_df = unknown_cat_df.rename(columns={'pred_Z': 'pred_Z_rAGN'})

Predicting redshift for rAGN in known dataset
Predicting redshift for rAGN in unknown dataset


In [47]:
if load_models_flag:
    print('Predicting redshift for rSFG in known dataset')
    known_catalog_df = gf.predict_z(known_catalog_df, redshift_reg_rGal)
    known_catalog_df = known_catalog_df.rename(columns={'pred_Z': 'pred_Z_rGal'})
    print('Predicting redshift for rSFG in unknown dataset')
    # unknown_cat_df = gf.predict_z(unknown_cat_df, redshift_reg_rGal)
    # unknown_cat_df = unknown_cat_df.rename(columns={'pred_Z': 'pred_Z_rGal'})

Predicting redshift for rSFG in known dataset
Predicting redshift for rSFG in unknown dataset


In [48]:
all_subsets_dfs      = [known_catalog_df, unknown_cat_df]

##### Include AGN detection criteria from literature

In [49]:
known_catalog_df = add_AGN_criteria(known_catalog_df)
known_catalog_df = add_AGN_criteria(known_catalog_df)

#### Only metrics for known sources predicted with calibrated models

In [50]:
np.array(known_catalog_df.columns)

array(['objID', 'RA_ICRS', 'DE_ICRS', 'Name', 'TYPE', 'Z', 'zsp', 'spCl',
       'band_num', 'LOFAR_detect', 'Sint_LOFAR', 'Sint_LOFAR_AB',
       'Speak_LOFAR', 'rms_LOFAR', 'Sint_LOFAR_non_imp',
       'Sint_LOFAR_AB_non_imp', 'Speak_LOFAR_non_imp', 'is_str',
       'is_SDSS_QSO', 'is_AGN', 'is_SDSS_gal', 'is_gal', 'W1mproPM',
       'W2mproPM', 'gmag', 'rmag', 'imag', 'zmag', 'ymag', 'W3mag',
       'W4mag', 'Jmag', 'Hmag', 'Kmag', 'g_r', 'g_i', 'g_z', 'g_y', 'g_J',
       'g_H', 'g_K', 'g_W1', 'g_W2', 'g_W3', 'g_W4', 'r_i', 'r_z', 'r_y',
       'r_J', 'r_H', 'r_K', 'r_W1', 'r_W2', 'r_W3', 'r_W4', 'i_z', 'i_y',
       'i_J', 'i_H', 'i_K', 'i_W1', 'i_W2', 'i_W3', 'i_W4', 'z_y', 'z_J',
       'z_H', 'z_K', 'z_W1', 'z_W2', 'z_W3', 'z_W4', 'y_J', 'y_H', 'y_K',
       'y_W1', 'y_W2', 'y_W3', 'y_W4', 'J_H', 'J_K', 'J_W1', 'J_W2',
       'J_W3', 'J_W4', 'H_K', 'H_W1', 'H_W2', 'H_W3', 'H_W4', 'K_W1',
       'K_W2', 'K_W3', 'K_W4', 'W1_W2', 'W1_W3', 'W1_W4', 'W2_W3',
       'W2_W4', 'W3_W4',

In [51]:
filt_true_AGN  = np.array(known_catalog_df.loc[:, 'class'] == 1)
filt_true_gal  = np.array(known_catalog_df.loc[:, 'class'] == 0)
filt_true_rAGN = np.array(known_catalog_df.loc[:, 'class'] == 1) & np.array(known_catalog_df.loc[:, 'LOFAR_detect'] == 1)
filt_true_rGal = np.array(known_catalog_df.loc[:, 'class'] == 0) & np.array(known_catalog_df.loc[:, 'LOFAR_detect'] == 1)

In [52]:
filt_pred_AGN  = np.array(known_catalog_df.loc[:, 'pred_class_cal'] == 1)
filt_pred_gal  = np.array(known_catalog_df.loc[:, 'pred_class_cal'] == 0)
filt_pred_rAGN = np.array(known_catalog_df.loc[:, 'pred_class_cal'] == 1) & np.array(known_catalog_df.loc[:, 'pred_radio_cal_AGN'] == 1)
filt_pred_rGal = np.array(known_catalog_df.loc[:, 'pred_class_cal'] == 0) & np.array(known_catalog_df.loc[:, 'pred_radio_cal_gal'] == 1)

In [53]:
cm_AGN        = gf.conf_mat_func(known_catalog_df.loc[:, 'class'],
                                 known_catalog_df.loc[:, 'pred_class_cal'])
cm_AGN_random = gf.conf_mat_random(known_catalog_df.loc[:, 'class'])

cm_gal        = np.rot90(cm_AGN, k=2)
cm_gal_random = np.rot90(cm_AGN_random, k=2)

In [54]:
cm_r_in_pred_AGN        = gf.conf_mat_func(known_catalog_df.loc[filt_pred_AGN, 'LOFAR_detect'],
                                           known_catalog_df.loc[filt_pred_AGN, 'pred_radio_cal_AGN'])
cm_r_in_true_AGN        = gf.conf_mat_func(known_catalog_df.loc[filt_true_AGN, 'LOFAR_detect'],
                                           known_catalog_df.loc[filt_true_AGN, 'pred_radio_cal_AGN'])
cm_r_in_pred_AGN_random = gf.conf_mat_random(known_catalog_df.loc[filt_pred_AGN, 'LOFAR_detect'])
cm_r_in_true_AGN_random = gf.conf_mat_random(known_catalog_df.loc[filt_true_AGN, 'LOFAR_detect'])

cm_r_in_pred_gal        = gf.conf_mat_func(known_catalog_df.loc[filt_pred_gal, 'LOFAR_detect'],
                                           known_catalog_df.loc[filt_pred_gal, 'pred_radio_cal_gal'])
cm_r_in_true_gal        = gf.conf_mat_func(known_catalog_df.loc[filt_true_gal, 'LOFAR_detect'],
                                           known_catalog_df.loc[filt_true_gal, 'pred_radio_cal_gal'])
cm_r_in_pred_gal_random = gf.conf_mat_random(known_catalog_df.loc[filt_pred_gal, 'LOFAR_detect'])
cm_r_in_true_gal_random = gf.conf_mat_random(known_catalog_df.loc[filt_true_gal, 'LOFAR_detect'])

In [55]:
cm_rAGN        = gf.conf_mat_func(known_catalog_df.loc[filt_pred_AGN, 'radio_AGN'],
                                  known_catalog_df.loc[filt_pred_AGN, 'pred_rAGN_cal'])
cm_rAGN_random = gf.conf_mat_random(known_catalog_df.loc[filt_pred_AGN, 'radio_AGN'])

cm_rGal        = gf.conf_mat_func(known_catalog_df.loc[filt_pred_gal, 'radio_gal'],
                                  known_catalog_df.loc[filt_pred_gal, 'pred_rGal_cal'])
cm_rGal_random = gf.conf_mat_random(known_catalog_df.loc[filt_pred_gal, 'radio_gal'])

Confusion matrices from literature AGN criteria.

In [56]:
cm_AGN_S12 = gf.conf_mat_func(known_catalog_df.loc[:, 'class'],
                              known_catalog_df.loc[:, 'S12_AGN'])
cm_AGN_M12 = gf.conf_mat_func(known_catalog_df.loc[:, 'class'],
                              known_catalog_df.loc[:, 'M12_AGN'])
cm_AGN_M16 = gf.conf_mat_func(known_catalog_df.loc[:, 'class'],
                              known_catalog_df.loc[:, 'M16_AGN'])
cm_AGN_B18 = gf.conf_mat_func(known_catalog_df.loc[:, 'class'],
                              known_catalog_df.loc[:, 'B18_AGN'])
cm_AGN_C22 = gf.conf_mat_func(known_catalog_df.loc[:, 'class'],
                              known_catalog_df.loc[:, 'C22_AGN'])

cm_gal_S12 = np.rot90(cm_AGN_S12, k=2)
cm_gal_M12 = np.rot90(cm_AGN_M12, k=2)
cm_gal_M16 = np.rot90(cm_AGN_M16, k=2)
cm_gal_B18 = np.rot90(cm_AGN_B18, k=2)
cm_gal_C22 = np.rot90(cm_AGN_C22, k=2)

Metrics from confusion matrices

In [57]:
MCC_AGN              = gf.MCC_from_CM(cm_AGN)
Fb_AGN               = gf.Fb_from_CM(cm_AGN)
Precision_AGN        = gf.Precision_from_CM(cm_AGN)
Recall_AGN           = gf.Recall_from_CM(cm_AGN)
ACC_AGN              = gf.ACC_from_CM(cm_AGN)

MCC_AGN_random       = gf.MCC_from_CM(cm_AGN_random)
ACC_AGN_random       = gf.ACC_from_CM(cm_AGN_random)
Fb_AGN_random        = gf.Fb_from_CM(cm_AGN_random)
Precision_AGN_random = gf.Precision_from_CM(cm_AGN_random)
Recall_AGN_random    = gf.Recall_from_CM(cm_AGN_random)

In [58]:
MCC_r_in_pred_AGN              = gf.MCC_from_CM(cm_r_in_pred_AGN)
Fb_r_in_pred_AGN               = gf.Fb_from_CM(cm_r_in_pred_AGN)
Precision_r_in_pred_AGN        = gf.Precision_from_CM(cm_r_in_pred_AGN)
Recall_r_in_pred_AGN           = gf.Recall_from_CM(cm_r_in_pred_AGN)
ACC_r_in_pred_AGN              = gf.ACC_from_CM(cm_r_in_pred_AGN)

MCC_r_in_true_AGN              = gf.MCC_from_CM(cm_r_in_true_AGN)
Fb_r_in_true_AGN               = gf.Fb_from_CM(cm_r_in_true_AGN)
Precision_r_in_true_AGN        = gf.Precision_from_CM(cm_r_in_true_AGN)
Recall_r_in_true_AGN           = gf.Recall_from_CM(cm_r_in_true_AGN)
ACC_r_in_true_AGN              = gf.ACC_from_CM(cm_r_in_true_AGN)

MCC_r_in_pred_AGN_random       = gf.MCC_from_CM(cm_r_in_pred_AGN_random)
ACC_r_in_pred_AGN_random       = gf.ACC_from_CM(cm_r_in_pred_AGN_random)
Fb_r_in_pred_AGN_random        = gf.Fb_from_CM(cm_r_in_pred_AGN_random)
Precision_r_in_pred_AGN_random = gf.Precision_from_CM(cm_r_in_pred_AGN_random)
Recall_r_in_pred_AGN_random    = gf.Recall_from_CM(cm_r_in_pred_AGN_random)

MCC_r_in_true_AGN_random       = gf.MCC_from_CM(cm_r_in_true_AGN_random)
ACC_r_in_true_AGN_random       = gf.ACC_from_CM(cm_r_in_true_AGN_random)
Fb_r_in_true_AGN_random        = gf.Fb_from_CM(cm_r_in_true_AGN_random)
Precision_r_in_true_AGN_random = gf.Precision_from_CM(cm_r_in_true_AGN_random)
Recall_r_in_true_AGN_random    = gf.Recall_from_CM(cm_r_in_true_AGN_random)

In [59]:
MCC_rAGN              = gf.MCC_from_CM(cm_rAGN)
Fb_rAGN               = gf.Fb_from_CM(cm_rAGN)
Precision_rAGN        = gf.Precision_from_CM(cm_rAGN)
Recall_rAGN           = gf.Recall_from_CM(cm_rAGN)
ACC_rAGN              = gf.ACC_from_CM(cm_rAGN)

MCC_rAGN_random       = gf.MCC_from_CM(cm_rAGN_random)
ACC_rAGN_random       = gf.ACC_from_CM(cm_rAGN_random)
Fb_rAGN_random        = gf.Fb_from_CM(cm_rAGN_random)
Precision_rAGN_random = gf.Precision_from_CM(cm_rAGN_random)
Recall_rAGN_random    = gf.Recall_from_CM(cm_rAGN_random)

In [60]:
MCC_gal              = gf.MCC_from_CM(cm_gal)
Fb_gal               = gf.Fb_from_CM(cm_gal)
Precision_gal        = gf.Precision_from_CM(cm_gal)
Recall_gal           = gf.Recall_from_CM(cm_gal)
ACC_gal              = gf.ACC_from_CM(cm_gal)

MCC_gal_random       = gf.MCC_from_CM(cm_gal_random)
ACC_gal_random       = gf.ACC_from_CM(cm_gal_random)
Fb_gal_random        = gf.Fb_from_CM(cm_gal_random)
Precision_gal_random = gf.Precision_from_CM(cm_gal_random)
Recall_gal_random    = gf.Recall_from_CM(cm_gal_random)

In [61]:
MCC_r_in_pred_gal              = gf.MCC_from_CM(cm_r_in_pred_gal)
Fb_r_in_pred_gal               = gf.Fb_from_CM(cm_r_in_pred_gal)
Precision_r_in_pred_gal        = gf.Precision_from_CM(cm_r_in_pred_gal)
Recall_r_in_pred_gal           = gf.Recall_from_CM(cm_r_in_pred_gal)
ACC_r_in_pred_gal              = gf.ACC_from_CM(cm_r_in_pred_gal)

MCC_r_in_true_gal              = gf.MCC_from_CM(cm_r_in_true_gal)
Fb_r_in_true_gal               = gf.Fb_from_CM(cm_r_in_true_gal)
Precision_r_in_true_gal        = gf.Precision_from_CM(cm_r_in_true_gal)
Recall_r_in_true_gal           = gf.Recall_from_CM(cm_r_in_true_gal)
ACC_r_in_true_gal              = gf.ACC_from_CM(cm_r_in_true_gal)

MCC_r_in_pred_gal_random       = gf.MCC_from_CM(cm_r_in_pred_gal_random)
ACC_r_in_pred_gal_random       = gf.ACC_from_CM(cm_r_in_pred_gal_random)
Fb_r_in_pred_gal_random        = gf.Fb_from_CM(cm_r_in_pred_gal_random)
Precision_r_in_pred_gal_random = gf.Precision_from_CM(cm_r_in_pred_gal_random)
Recall_r_in_pred_gal_random    = gf.Recall_from_CM(cm_r_in_pred_gal_random)

MCC_r_in_true_gal_random       = gf.MCC_from_CM(cm_r_in_true_gal_random)
ACC_r_in_true_gal_random       = gf.ACC_from_CM(cm_r_in_true_gal_random)
Fb_r_in_true_gal_random        = gf.Fb_from_CM(cm_r_in_true_gal_random)
Precision_r_in_true_gal_random = gf.Precision_from_CM(cm_r_in_true_gal_random)
Recall_r_in_true_gal_random    = gf.Recall_from_CM(cm_r_in_true_gal_random)

In [62]:
MCC_rGal              = gf.MCC_from_CM(cm_rGal)
Fb_rGal               = gf.Fb_from_CM(cm_rGal)
Precision_rGal        = gf.Precision_from_CM(cm_rGal)
Recall_rGal           = gf.Recall_from_CM(cm_rGal)
ACC_rGal              = gf.ACC_from_CM(cm_rGal)

MCC_rGal_random       = gf.MCC_from_CM(cm_rGal_random)
ACC_rGal_random       = gf.ACC_from_CM(cm_rGal_random)
Fb_rGal_random        = gf.Fb_from_CM(cm_rGal_random)
Precision_rGal_random = gf.Precision_from_CM(cm_rGal_random)
Recall_rGal_random    = gf.Recall_from_CM(cm_rGal_random)

Confusion matrices for AGN criteria from literature

In [63]:
MCC_AGN_S12          = gf.MCC_from_CM(cm_AGN_S12)
ACC_AGN_S12          = gf.ACC_from_CM(cm_AGN_S12)
Fb_AGN_S12           = gf.Fb_from_CM(cm_AGN_S12)
Precision_AGN_S12    = gf.Precision_from_CM(cm_AGN_S12)
Recall_AGN_S12       = gf.Recall_from_CM(cm_AGN_S12)

MCC_AGN_M12          = gf.MCC_from_CM(cm_AGN_M12)
ACC_AGN_M12          = gf.ACC_from_CM(cm_AGN_M12)
Fb_AGN_M12           = gf.Fb_from_CM(cm_AGN_M12)
Precision_AGN_M12    = gf.Precision_from_CM(cm_AGN_M12)
Recall_AGN_M12       = gf.Recall_from_CM(cm_AGN_M12)

MCC_AGN_M16          = gf.MCC_from_CM(cm_AGN_M16)
ACC_AGN_M16          = gf.ACC_from_CM(cm_AGN_M16)
Fb_AGN_M16           = gf.Fb_from_CM(cm_AGN_M16)
Precision_AGN_M16    = gf.Precision_from_CM(cm_AGN_M16)
Recall_AGN_M16       = gf.Recall_from_CM(cm_AGN_M16)

MCC_AGN_B18          = gf.MCC_from_CM(cm_AGN_B18)
ACC_AGN_B18          = gf.ACC_from_CM(cm_AGN_B18)
Fb_AGN_B18           = gf.Fb_from_CM(cm_AGN_B18)
Precision_AGN_B18    = gf.Precision_from_CM(cm_AGN_B18)
Recall_AGN_B18       = gf.Recall_from_CM(cm_AGN_B18)

MCC_AGN_C22          = gf.MCC_from_CM(cm_AGN_C22)
ACC_AGN_C22          = gf.ACC_from_CM(cm_AGN_C22)
Fb_AGN_C22           = gf.Fb_from_CM(cm_AGN_C22)
Precision_AGN_C22    = gf.Precision_from_CM(cm_AGN_C22)
Recall_AGN_C22       = gf.Recall_from_CM(cm_AGN_C22)

metrics_AGN_criteria    = np.array([[Fb_AGN_S12,        Fb_AGN_M12,        Fb_AGN_M16,        Fb_AGN_B18,        Fb_AGN_C22],\
                                    [MCC_AGN_S12,       MCC_AGN_M12,       MCC_AGN_M16,       MCC_AGN_B18,       MCC_AGN_C22],\
                                    [Precision_AGN_S12, Precision_AGN_M12, Precision_AGN_M16, Precision_AGN_B18, Precision_AGN_C22],\
                                    [Recall_AGN_S12,    Recall_AGN_M12,    Recall_AGN_M16,    Recall_AGN_B18,    Recall_AGN_C22],\
                                    [ACC_AGN_S12,       ACC_AGN_M12,       ACC_AGN_M16,       ACC_AGN_B18,       ACC_AGN_C22]]) 

metrics_AGN_criteria_df = pd.DataFrame(data=metrics_AGN_criteria.T, columns=['F-\u03B2', 'MCC', 'Precision', 'Recall', 'Accuracy'],\
                                       index=['S12', 'M12', 'M16', 'B18', 'C22'])

Metrics from redshift predictions

In [64]:
sigma_mad_in_pred_rAGN    = gf.sigma_mad(known_catalog_df.loc[filt_pred_rAGN, 'Z'],    known_catalog_df.loc[filt_pred_rAGN, 'pred_Z_rAGN'])
sigma_nmad_in_pred_rAGN   = gf.sigma_nmad(known_catalog_df.loc[filt_pred_rAGN, 'Z'],   known_catalog_df.loc[filt_pred_rAGN, 'pred_Z_rAGN'])
sigma_z_in_pred_rAGN      = gf.sigma_z(known_catalog_df.loc[filt_pred_rAGN, 'Z'],      known_catalog_df.loc[filt_pred_rAGN, 'pred_Z_rAGN'])
sigma_z_norm_in_pred_rAGN = gf.sigma_z_norm(known_catalog_df.loc[filt_pred_rAGN, 'Z'], known_catalog_df.loc[filt_pred_rAGN, 'pred_Z_rAGN'])
out_frac_in_pred_rAGN     = gf.outlier_frac(known_catalog_df.loc[filt_pred_rAGN, 'Z'], known_catalog_df.loc[filt_pred_rAGN, 'pred_Z_rAGN'])

sigma_mad_in_true_rAGN    = gf.sigma_mad(known_catalog_df.loc[filt_true_rAGN, 'Z'],    known_catalog_df.loc[filt_true_rAGN, 'pred_Z_rAGN'])
sigma_nmad_in_true_rAGN   = gf.sigma_nmad(known_catalog_df.loc[filt_true_rAGN, 'Z'],   known_catalog_df.loc[filt_true_rAGN, 'pred_Z_rAGN'])
sigma_z_in_true_rAGN      = gf.sigma_z(known_catalog_df.loc[filt_true_rAGN, 'Z'],      known_catalog_df.loc[filt_true_rAGN, 'pred_Z_rAGN'])
sigma_z_norm_in_true_rAGN = gf.sigma_z_norm(known_catalog_df.loc[filt_true_rAGN, 'Z'], known_catalog_df.loc[filt_true_rAGN, 'pred_Z_rAGN'])
out_frac_in_true_rAGN     = gf.outlier_frac(known_catalog_df.loc[filt_true_rAGN, 'Z'], known_catalog_df.loc[filt_true_rAGN, 'pred_Z_rAGN'])

In [65]:
sigma_mad_in_pred_rGal    = gf.sigma_mad(known_catalog_df.loc[filt_pred_rGal, 'Z'],    known_catalog_df.loc[filt_pred_rGal, 'pred_Z_rGal'])
sigma_nmad_in_pred_rGal   = gf.sigma_nmad(known_catalog_df.loc[filt_pred_rGal, 'Z'],   known_catalog_df.loc[filt_pred_rGal, 'pred_Z_rGal'])
sigma_z_in_pred_rGal      = gf.sigma_z(known_catalog_df.loc[filt_pred_rGal, 'Z'],      known_catalog_df.loc[filt_pred_rGal, 'pred_Z_rGal'])
sigma_z_norm_in_pred_rGal = gf.sigma_z_norm(known_catalog_df.loc[filt_pred_rGal, 'Z'], known_catalog_df.loc[filt_pred_rGal, 'pred_Z_rGal'])
out_frac_in_pred_rGal     = gf.outlier_frac(known_catalog_df.loc[filt_pred_rGal, 'Z'], known_catalog_df.loc[filt_pred_rGal, 'pred_Z_rGal'])

sigma_mad_in_true_rGal    = gf.sigma_mad(known_catalog_df.loc[filt_true_rGal, 'Z'],    known_catalog_df.loc[filt_true_rGal, 'pred_Z_rGal'])
sigma_nmad_in_true_rGal   = gf.sigma_nmad(known_catalog_df.loc[filt_true_rGal, 'Z'],   known_catalog_df.loc[filt_true_rGal, 'pred_Z_rGal'])
sigma_z_in_true_rGal      = gf.sigma_z(known_catalog_df.loc[filt_true_rGal, 'Z'],      known_catalog_df.loc[filt_true_rGal, 'pred_Z_rGal'])
sigma_z_norm_in_true_rGal = gf.sigma_z_norm(known_catalog_df.loc[filt_true_rGal, 'Z'], known_catalog_df.loc[filt_true_rGal, 'pred_Z_rGal'])
out_frac_in_true_rGal     = gf.outlier_frac(known_catalog_df.loc[filt_true_rGal, 'Z'], known_catalog_df.loc[filt_true_rGal, 'pred_Z_rGal'])

Obtain uncertainties from CV

In [66]:
subsets_clf_filt    = [np.array([True] * len(known_catalog_df)), np.array([True] * len(known_catalog_df)),
                       filt_true_AGN, filt_pred_AGN, filt_true_gal, filt_pred_gal,
                       np.array([True] * len(known_catalog_df)), np.array([True] * len(known_catalog_df))]
subsets_reg_filt    = [filt_true_rAGN, filt_pred_rAGN, filt_true_rGal, filt_pred_rGal]

In [67]:
subsets_clf_names  = ['AGN', 'Galaxy', 'Radio in true AGN', 'Radio in pred AGN',
                      'Radio in true gal', 'Radio in pred gal', 'Radio AGN', 'Radio gal']
subsets_reg_names  = ['Redshift in true rAGN', 'Redshift in pred rAGN',
                      'Redshift in true rGal', 'Redshift in pred rGal']

In [68]:
true_classes_clf   = ['class', 'class_gal',
                      'LOFAR_detect', 'LOFAR_detect', 'LOFAR_detect', 'LOFAR_detect',
                      'radio_AGN', 'radio_gal']
pred_classes_clf   = ['pred_class_cal', 'pred_class_gal_cal', 'pred_radio_cal_AGN',
                      'pred_radio_cal_AGN', 'pred_radio_cal_gal', 'pred_radio_cal_gal',
                      'pred_rAGN_cal', 'pred_rGal_cal']
true_target_reg    = ['Z', 'Z', 'Z', 'Z']
pred_target_reg    = ['pred_Z_rAGN', 'pred_Z_rAGN', 'pred_Z_rGal', 'pred_Z_rGal']

In [69]:
clf_scores_names   = ['F-\u03B2', 'MCC', 'Precision', 'Recall']
reg_scores_names   = ['\u03C3 MAD', '\u03C3 NMAD', '\u03C3 z', '\u03C3 z N', '\u03B7']

In [70]:
CV_object_clf      = StratifiedKFold(n_splits=10, random_state=gv.seed, shuffle=True)
CV_object_reg      = KFold(n_splits=10, random_state=gv.seed, shuffle=True)

In [71]:
cv_scores_clf      = {}
cv_scores_mean_clf = {}
cv_scores_std_clf  = {}

In [72]:
cv_scores_reg      = {}
cv_scores_mean_reg = {}
cv_scores_std_reg  = {}

In [73]:
for count, (subset_filter, subset_name) in enumerate(zip(subsets_clf_filt, subsets_clf_names)):
    tmp_f_betas = []
    tmp_mccs    = []
    tmp_precs   = []
    tmp_recalls = []
    for _, fold_index in iter(CV_object_clf.split(known_catalog_df.loc[subset_filter,
                                                  pred_classes_clf[count]],
                                                  known_catalog_df.loc[subset_filter, true_classes_clf[count]])):
        tmp_cm     = gf.conf_mat_func(known_catalog_df.loc[subset_filter, true_classes_clf[count]].iloc[fold_index], 
                                      known_catalog_df.loc[subset_filter, pred_classes_clf[count]].iloc[fold_index])
        tmp_f_beta = gf.Fb_from_CM(tmp_cm)
        tmp_mcc    = gf.MCC_from_CM(tmp_cm)
        tmp_prec   = gf.Precision_from_CM(tmp_cm)
        tmp_recall = gf.Recall_from_CM(tmp_cm)
        tmp_f_betas.append(tmp_f_beta)
        tmp_mccs.append(tmp_mcc)
        tmp_precs.append(tmp_prec)
        tmp_recalls.append(tmp_recall)
    cv_scores_clf[subset_name] = {'F-\u03B2': tmp_f_betas, 'MCC': tmp_mccs, 
                                  'Precision': tmp_precs, 'Recall': tmp_recalls}

In [74]:
for count, (subset_filter, subset_name) in enumerate(zip(subsets_reg_filt, subsets_reg_names)):
    tmp_s_mads  = []
    tmp_s_nmads = []
    tmp_s_zs    = []
    tmp_s_z_ns  = []
    tmp_out_fs  = []
    for _, fold_index in iter(CV_object_reg.split(known_catalog_df.loc[subset_filter, pred_target_reg[count]],
                                                  known_catalog_df.loc[subset_filter, true_target_reg[count]])):
        tmp_s_mad  = gf.sigma_mad(known_catalog_df.loc[subset_filter, true_target_reg[count]].iloc[fold_index], 
                                  known_catalog_df.loc[subset_filter, pred_target_reg[count]].iloc[fold_index])
        tmp_s_nmad = gf.sigma_nmad(known_catalog_df.loc[subset_filter, true_target_reg[count]].iloc[fold_index],
                                   known_catalog_df.loc[subset_filter, pred_target_reg[count]].iloc[fold_index])
        tmp_s_z    = gf.sigma_z(known_catalog_df.loc[subset_filter, true_target_reg[count]].iloc[fold_index], 
                                known_catalog_df.loc[subset_filter, pred_target_reg[count]].iloc[fold_index])
        tmp_s_z_n  = gf.sigma_z_norm(known_catalog_df.loc[subset_filter, true_target_reg[count]].iloc[fold_index], 
                                     known_catalog_df.loc[subset_filter, pred_target_reg[count]].iloc[fold_index])
        tmp_out_f  = gf.outlier_frac(known_catalog_df.loc[subset_filter, true_target_reg[count]].iloc[fold_index], 
                                     known_catalog_df.loc[subset_filter, pred_target_reg[count]].iloc[fold_index])
        tmp_s_mads.append(tmp_s_mad)
        tmp_s_nmads.append(tmp_s_nmad)
        tmp_s_zs.append(tmp_s_z)
        tmp_s_z_ns.append(tmp_s_z_n)
        tmp_out_fs.append(tmp_out_f)
    cv_scores_reg[subset_name] = {'\u03C3 MAD': tmp_s_mads, '\u03C3 NMAD': tmp_s_nmads,
                                  '\u03C3 z': tmp_s_zs, '\u03C3 z N': tmp_s_z_ns, 
                                  '\u03B7':tmp_out_fs}

In [75]:
for sub_set in cv_scores_clf:
    cv_scores_mean_clf[sub_set] = {}
    cv_scores_std_clf[sub_set]  = {}
    for score in cv_scores_clf[sub_set]:
        cv_scores_mean_clf[sub_set][score] = np.nanmean(cv_scores_clf[sub_set][score])
        cv_scores_std_clf[sub_set][score]  = np.nanstd(cv_scores_clf[sub_set][score])

In [76]:
for sub_set in cv_scores_reg:
    cv_scores_mean_reg[sub_set] = {}
    cv_scores_std_reg[sub_set]  = {}
    for score in cv_scores_reg[sub_set]:
        cv_scores_mean_reg[sub_set][score] = np.nanmean(cv_scores_reg[sub_set][score])
        cv_scores_std_reg[sub_set][score]  = np.nanstd(cv_scores_reg[sub_set][score])

In [77]:
cv_scores_mean_clf_df = pd.DataFrame(columns=clf_scores_names, index=[*cv_scores_mean_clf.keys()])
cv_scores_std_clf_df  = pd.DataFrame(columns=clf_scores_names, index=[*cv_scores_std_clf.keys()])

cv_scores_mean_reg_df = pd.DataFrame(columns=reg_scores_names, index=[*cv_scores_mean_reg.keys()])
cv_scores_std_reg_df  = pd.DataFrame(columns=reg_scores_names, index=[*cv_scores_std_reg.keys()])

In [78]:
for row in [*cv_scores_mean_clf.keys()]:
    for column in [*cv_scores_mean_clf[row].keys()]:
        cv_scores_mean_clf_df.loc[row, column] = cv_scores_mean_clf[row][column]

In [79]:
for row in [*cv_scores_std_clf.keys()]:
    for column in [*cv_scores_std_clf[row].keys()]:
        cv_scores_std_clf_df.loc[row, column] = cv_scores_std_clf[row][column]

In [80]:
for row in [*cv_scores_mean_reg.keys()]:
    for column in [*cv_scores_mean_reg[row].keys()]:
        cv_scores_mean_reg_df.loc[row, column] = cv_scores_mean_reg[row][column]

In [81]:
for row in [*cv_scores_std_reg.keys()]:
    for column in [*cv_scores_std_reg[row].keys()]:
        cv_scores_std_reg_df.loc[row, column] = cv_scores_std_reg[row][column]

Join metrics for classification.

In [85]:
metrics_classif      = np.array([[len(known_catalog_df), Fb_AGN,       MCC_AGN,       Precision_AGN,       Recall_AGN,       ACC_AGN],
                                 [len(known_catalog_df), Fb_gal,       MCC_gal,       Precision_gal,       Recall_gal,       ACC_gal],
                                 [np.sum(filt_true_AGN),  Fb_r_in_true_AGN, MCC_r_in_true_AGN, Precision_r_in_true_AGN, Recall_r_in_true_AGN, ACC_r_in_true_AGN],
                                 [np.sum(filt_pred_AGN),  Fb_r_in_pred_AGN, MCC_r_in_pred_AGN, Precision_r_in_pred_AGN, Recall_r_in_pred_AGN, ACC_r_in_pred_AGN],
                                 [np.sum(filt_true_gal), Fb_r_in_true_gal, MCC_r_in_true_gal, Precision_r_in_true_gal, Recall_r_in_true_gal, ACC_r_in_true_gal],
                                 [np.sum(filt_pred_gal), Fb_r_in_pred_gal, MCC_r_in_pred_gal, Precision_r_in_pred_gal, Recall_r_in_pred_gal, ACC_r_in_pred_gal],
                                 [len(known_catalog_df), Fb_rAGN, MCC_rAGN, Precision_rAGN, Recall_rAGN, ACC_rAGN],
                                 [len(known_catalog_df), Fb_rGal, MCC_rGal, Precision_rGal, Recall_rGal, ACC_rGal]])
metrics_classif_df   = pd.DataFrame(data=metrics_classif, columns=['Sample', 'F-\u03B2', 'MCC', 'Precision', 'Recall', 'Accuracy'],
                                    index=subsets_clf_names)

metrics_classif_random    = np.array([[len(known_catalog_df), Fb_AGN_random,       MCC_AGN_random,       Precision_AGN_random,       Recall_AGN_random,       ACC_AGN_random],
                                 [len(known_catalog_df), Fb_gal_random,       MCC_gal_random,       Precision_gal_random,       Recall_gal_random,       ACC_gal_random],
                                 [np.sum(filt_true_AGN),  Fb_r_in_true_AGN_random, MCC_r_in_true_AGN_random, Precision_r_in_true_AGN_random, Recall_r_in_true_AGN_random, ACC_r_in_true_AGN_random],
                                 [np.sum(filt_pred_AGN),  Fb_r_in_pred_AGN_random, MCC_r_in_pred_AGN_random, Precision_r_in_pred_AGN_random, Recall_r_in_pred_AGN_random, ACC_r_in_pred_AGN_random],
                                 [np.sum(filt_true_gal), Fb_r_in_true_gal_random, MCC_r_in_true_gal_random, Precision_r_in_true_gal_random, Recall_r_in_true_gal_random, ACC_r_in_true_gal_random],
                                 [np.sum(filt_pred_gal), Fb_r_in_pred_gal_random, MCC_r_in_pred_gal_random, Precision_r_in_pred_gal_random, Recall_r_in_pred_gal_random, ACC_r_in_pred_gal_random],
                                 [len(known_catalog_df), Fb_rAGN_random, MCC_rAGN_random, Precision_rAGN_random, Recall_rAGN_random, ACC_rAGN_random],
                                 [len(known_catalog_df), Fb_rGal_random, MCC_rGal_random, Precision_rGal_random, Recall_rGal_random, ACC_rGal_random]])

metrics_classif_random_df   = pd.DataFrame(data=metrics_classif_random, 
                                           columns=['Sample', 'F-\u03B2', 'MCC', 'Precision', 'Recall', 'Accuracy'], 
                                           index=subsets_clf_names)

Join metrics for regression.

In [89]:
size_true_rAGN    = np.sum(filt_true_rAGN)
size_true_rGal    = np.sum(filt_true_rGal)
size_pred_rAGN    = np.sum(filt_pred_rAGN)
size_pred_rGal    = np.sum(filt_pred_rGal)
metrics_z    = np.array([[size_true_rAGN, sigma_mad_in_true_rAGN, sigma_nmad_in_true_rAGN, sigma_z_in_true_rAGN, sigma_z_norm_in_true_rAGN, out_frac_in_true_rAGN],
                         [size_pred_rAGN, sigma_mad_in_pred_rAGN, sigma_nmad_in_pred_rAGN, sigma_z_in_pred_rAGN, sigma_z_norm_in_pred_rAGN, out_frac_in_pred_rAGN],
                         [size_true_rGal, sigma_mad_in_true_rGal, sigma_nmad_in_true_rGal, sigma_z_in_true_rGal, sigma_z_norm_in_true_rGal, out_frac_in_true_rGal],
                         [size_pred_rGal, sigma_mad_in_pred_rGal, sigma_nmad_in_pred_rGal, sigma_z_in_pred_rGal, sigma_z_norm_in_pred_rGal, out_frac_in_pred_rGal]])
metrics_z_df = pd.DataFrame(data=metrics_z,
                            columns=['Sample', '\u03C3 MAD', '\u03C3 NMAD', '\u03C3 z', '\u03C3 z N', '\u03B7'],
                            index=subsets_reg_names)

Print metrics

In [94]:
print('Metrics for classification steps.')
metrics_classif_df.loc[:, 'Sample'] = metrics_classif_df.loc[:, 'Sample'].astype(int)
with pd.option_context('display.float_format', '{:.4f}'.format):
    display(metrics_classif_df.drop(columns=['Accuracy']))

Metrics for classification steps.


,Sample,F-β,MCC,Precision,Recall
AGN,23747,0.9542,0.9185,0.9449,0.9621
Galaxy,23747,0.9643,0.9185,0.9715,0.9584
Radio in true AGN,10108,0.4648,0.3484,0.4332,0.4946
Radio in pred AGN,10292,0.4481,0.3380,0.4223,0.4718
Radio in true gal,13639,0.4587,0.3662,0.4612,0.4566
Radio in pred gal,13455,0.4619,0.3667,0.4703,0.4551
Radio AGN,23747,0.4228,0.3115,0.3870,0.4579
Radio gal,23747,0.4331,0.3355,0.4224,0.4423


In [96]:
print('Metrics and uncertainties for classification using CV.')
for df_object in [cv_scores_mean_clf_df, cv_scores_std_clf_df]:
    df_object.loc[:, 'Sample'] = metrics_classif_df.loc[df_object.index, 'Sample']
with pd.option_context('display.float_format', '{:.4f}'.format):
    display(cv_scores_mean_clf_df.loc[:, ['Sample', 'F-\u03B2', 'MCC', 'Precision', 'Recall']])
    display(cv_scores_std_clf_df.loc[:, ['Sample', 'F-\u03B2', 'MCC', 'Precision', 'Recall']])

Metrics and uncertainties for classification using CV.


,Sample,F-β,MCC,Precision,Recall
AGN,23747,0.9543,0.9185,0.9450,0.9621
Galaxy,23747,0.9643,0.9185,0.9715,0.9584
Radio in true AGN,10108,0.4646,0.3486,0.4337,0.4946
Radio in pred AGN,10292,0.4482,0.3385,0.4234,0.4719
Radio in true gal,13639,0.4588,0.3667,0.4623,0.4566
Radio in pred gal,13455,0.4622,0.3673,0.4717,0.4551
Radio AGN,23747,0.4019,0.3542,0.3881,0.4147
Radio gal,23747,0.4224,0.3694,0.4228,0.4226


,Sample,F-β,MCC,Precision,Recall
AGN,23747,0.0038,0.0070,0.0065,0.0043
Galaxy,23747,0.0033,0.0070,0.0031,0.0052
Radio in true AGN,10108,0.0325,0.0390,0.0318,0.0395
Radio in pred AGN,10292,0.0297,0.0367,0.0327,0.0328
Radio in true gal,13639,0.0254,0.0308,0.0321,0.0262
Radio in pred gal,13455,0.0311,0.0389,0.0394,0.0271
Radio AGN,23747,0.0285,0.0313,0.0322,0.0296
Radio gal,23747,0.0219,0.0232,0.0216,0.0277


In [97]:
print('Metrics for classification steps using random guesses to obtain classes.')
metrics_classif_random_df.loc[:, 'Sample'] = metrics_classif_random_df.loc[:, 'Sample'].astype(int)
with pd.option_context('display.float_format', '{:.4f}'.format):
    display(metrics_classif_random_df.drop(columns=['Accuracy']))

Metrics for classification steps using random guesses to obtain classes.


,Sample,F-β,MCC,Precision,Recall
AGN,23747,0.4257,0.0000,0.4257,0.4257
Galaxy,23747,0.5743,0.0000,0.5743,0.5743
Radio in true AGN,10108,0.1646,0.0000,0.1646,0.1646
Radio in pred AGN,10292,0.1551,0.0000,0.1551,0.1551
Radio in true gal,13639,0.1469,0.0000,0.1469,0.1469
Radio in pred gal,13455,0.1540,0.0000,0.1540,0.1540
Radio AGN,23747,0.1464,0.0000,0.1464,0.1464
Radio gal,23747,0.1423,0.0000,0.1423,0.1423


In [99]:
print('Metrics for AGN diagnostics criteria (from literature).')
print(f'Sample size, N = {np.sum(filter_known_spec):,}')
with pd.option_context('display.float_format', '{:.4f}'.format):
    display(metrics_AGN_criteria_df.drop(columns=['Accuracy']))

Metrics for AGN diagnostics criteria (from literature).
Sample size, N = 118,734


,F-β,MCC,Precision,Recall
S12,0.8610,0.7878,0.9398,0.8051
M12,0.5180,0.4971,0.9887,0.3718
M16,0.6721,0.6130,0.9748,0.5348
B18,0.8214,0.7576,0.9754,0.7266
C22,0.9271,0.8764,0.9400,0.9167


In [101]:
print('Metrics for redshift predictions for sources predicted to be detected in radio')
metrics_z_df.loc[:, 'Sample'] = metrics_z_df.loc[:, 'Sample'].astype(int)
with pd.option_context('display.float_format', '{:.4f}'.format):
    display(metrics_z_df)

Metrics for redshift predictions for sources predicted to be detected in radio


,Sample,σ MAD,σ NMAD,σ z,σ z N,η
Redshift in true rAGN,1664,0.1482,0.0664,0.3890,0.1820,0.1659
Redshift in pred rAGN,1783,0.1424,0.0656,0.3731,0.2380,0.1699
Redshift in true rGal,2004,0.0386,0.0289,0.1044,0.0709,0.0279
Redshift in pred rGal,2005,0.0355,0.0283,0.1622,0.0774,0.0279


In [103]:
print('Metrics and uncertainties for redshift predictions (from CV) for sources predicted to be detected in radio')
for df_object in [cv_scores_mean_reg_df, cv_scores_std_reg_df]:
    df_object.loc[:, 'Sample'] = metrics_z_df.loc[df_object.index, 'Sample']
with pd.option_context('display.float_format', '{:.4f}'.format):
    display(cv_scores_mean_reg_df.loc[:, ['Sample', '\u03C3 MAD', '\u03C3 NMAD', '\u03C3 z', '\u03C3 z N', '\u03B7']])
    display(cv_scores_std_reg_df.loc[:, ['Sample', '\u03C3 MAD', '\u03C3 NMAD', '\u03C3 z', '\u03C3 z N', '\u03B7']])

Metrics and uncertainties for redshift predictions (from CV) for sources predicted to be detected in radio


,Sample,σ MAD,σ NMAD,σ z,σ z N,η
Redshift in true rAGN,1664,0.1478,0.0665,0.3863,0.1797,0.1658
Redshift in pred rAGN,1783,0.1456,0.0665,0.3685,0.2304,0.1700
Redshift in true rGal,2004,0.0390,0.0293,0.1038,0.0701,0.0279
Redshift in pred rGal,2005,0.0355,0.0285,0.1435,0.0757,0.0279


,Sample,σ MAD,σ NMAD,σ z,σ z N,η
Redshift in true rAGN,1664,0.0209,0.0070,0.0454,0.0290,0.0272
Redshift in pred rAGN,1783,0.0234,0.0065,0.0590,0.0600,0.0308
Redshift in true rGal,2004,0.0034,0.0025,0.0106,0.0103,0.0097
Redshift in pred rGal,2005,0.0034,0.0030,0.0755,0.0161,0.0107


---